# Matching genes to proximal peaks

A common approach to start peak - gene correlation analysis is to find all peaks within 50kb of a gene. Here we build an adjacency matrix matching peak to genes.

In [1]:
import numpy as np
import scanpy as sc 
import pandas as pd
import anndata
import anndata2ri ## For sparse matrix conversion from r 2 py

#### r2py set-up

In [2]:
import rpy2.rinterface_lib.callbacks
import logging

In [3]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

Loading the `rpy2` extension enables cell magic to be used. This runs R code in jupyter notebook cells.

In [4]:
%load_ext rpy2.ipython

### Load data

In [5]:
outdir = "/home/ubuntu/experiments/heart/scATACseq/ed6_pipeline/regions/LV/"
experiment_prefix = 'hca_heart_LV'

In [6]:
adata = sc.read_h5ad(outdir + experiment_prefix + "_ATAC.wCisTopic.h5ad")
adata

AnnData object with n_obs × n_vars = 10145 × 135228
    obs: 'cellatac_clusters', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'peak_width', 'exon', 'gene', 'promoter', 'annotation', 'gene_name', 'gene_id', 'tss_distance', 'ENCODE_blacklist', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'cellatac_clusters_colors', 'neighbors', 'umap'
    obsm: 'X_cistopic', 'X_umap'
    layers: 'binary_raw', 'cistopic_p'
    obsp: 'connectivities', 'distances'

In [7]:
peaks = adata.var_names

### Match peaks 2 genes

In [9]:
%%R 
library(Matrix)
library(GenomicRanges)
library(ensembldb)
library(EnsDb.Hsapiens.v86) ## Remember to pick your genome!
library(tidyr)
# library(Signac)

In [10]:
%%R
## String - GRanges conversion
## Borrowed from Signac functions 
## https://satijalab.org/signac/reference/GRangesToString.html
StringToGRanges <- function(regions, sep = c("-", "-"), ...) {
  ranges.df <- data.frame(ranges = regions)
  ranges.df <- separate(
    data = ranges.df,
    col = "ranges",
    sep = paste0(sep[[1]], "|", sep[[2]]),
    into = c("chr", "start", "end")
  )
  granges <- makeGRangesFromDataFrame(df = ranges.df, ...)
  return(granges)
}

GRangesToString <- function(grange, sep = c("-", "-")) {
  regions <- paste0(
    as.character(x = seqnames(x = grange)),
    sep[[1]],
    start(x = grange),
    sep[[2]],
    end(x = grange)
  )
  return(regions)
}

# Extend genomicRanges
# 
extend <- function(x, upstream=0, downstream=0)     
{
    if (any(strand(x) == "*"))
        warning("'*' ranges were treated as '+'")
    on_plus <- strand(x) == "+" | strand(x) == "*"
    new_start <- start(x) - ifelse(on_plus, upstream, downstream)
    new_end <- end(x) + ifelse(on_plus, downstream, upstream)
    ranges(x) <- IRanges(new_start, new_end)
    trim(x)
}


# Find peaks close to features of interest
#
# @param peaks_gr GenomicRanges object containing peaks
# @param features_gr GenomicRanges object containing features (e.g. genes)
# @param d distance to include peak, in bps (default 50000)
# @param feat_anno column in `features_gr@elementMetadata` containing annotation to name features (if NULL converts Granges to string)
#
# @return Sparse adjacency matrix indicating hits
peak2feature <- function(peaks_gr, features_gr, d=50000, feat_anno=NULL){
  seqlevelsStyle(features_gr) <- seqlevelsStyle(peaks_gr)
  
  ## Find peaks overlapping the search range around the features
  ext_gr <- extend(features_gr, upstream = d, downstream = d)
  ovs <- findOverlaps(peaks_gr, ext_gr)
  
  ## Define identifiers for peaks and features
  all_peaks <- GRangesToString(peaks_gr, sep = c(":", '-'))
  if (is.null(feat_anno)) {
    all_feats <- GRangesToString(features_gr, sep = c(":", '-'))
  } else {
    all_feats <- features_gr@elementMetadata[[feat_anno]]
  }
  
  ## Build adjacency matrix for hits
  adj_mat <- Matrix(data=0, nrow = length(all_peaks), ncol=length(all_feats))
  for (i in unique(subjectHits(ovs))) {
    # if (length(adj_mat[queryHits(ovs[subjectHits(ovs)==i]),i]) > 0) {
    adj_mat[queryHits(ovs[subjectHits(ovs)==i]),i] <- 1
    # }
  }
  colnames(adj_mat) <- all_feats
  rownames(adj_mat) <- all_peaks
  
  adj_mat
  
}

In [11]:
%%R  -i peaks -o adj_mat
genes_gr <- genes(EnsDb.Hsapiens.v86)
peaks_gr <- StringToGRanges(peaks, sep=c(":", "-"))

## Compute peak2gene adjacency matrix
adj_mat <- peak2feature(peaks_gr, genes_gr, feat_anno = "gene_id", d=50000)

In [12]:
%%R -o genes
genes <- colnames(adj_mat)

In [13]:
## Convert sparse matrix w anndata2ri
adj_mat = anndata2ri.r2py.rmat_to_spmat(adj_mat)

We can store the adjacency matrix in the `.varm` slot of the anndata

In [14]:
adata.varm["peak2gene"] = adj_mat
adata.uns["peak2gene_genes"] = genes

In [15]:
adata

AnnData object with n_obs × n_vars = 10145 × 135228
    obs: 'cellatac_clusters', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'peak_width', 'exon', 'gene', 'promoter', 'annotation', 'gene_name', 'gene_id', 'tss_distance', 'ENCODE_blacklist', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'cellatac_clusters_colors', 'neighbors', 'umap', 'peak2gene_genes'
    obsm: 'X_cistopic', 'X_umap'
    varm: 'peak2gene'
    layers: 'binary_raw', 'cistopic_p'
    obsp: 'connectivities', 'distances'

In [17]:
adata.var.head()

,peak_width,exon,gene,promoter,annotation,gene_name,gene_id,tss_distance,ENCODE_blacklist,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts
peak_id,,,,,,,,,,,,,,,
chr1:816981-817435,455,0,0,0,intergenic,nan,nan,106492,0,563,0.053891,0.052489,94.610893,563.0,6.335054
chr1:831286-831573,288,0,0,0,intergenic,nan,nan,92354,0,474,0.045372,0.044373,95.462812,474.0,6.163315
chr1:832024-832684,661,0,0,0,intergenic,nan,nan,91243,0,922,0.088255,0.084575,91.174500,922.0,6.827629
chr1:834224-834560,337,0,0,0,intergenic,nan,nan,89367,0,345,0.033024,0.032490,96.697617,345.0,5.846439
chr1:835550-835763,214,0,0,0,intergenic,nan,nan,88164,0,553,0.052934,0.051580,94.706614,553.0,6.317165


### Save anndata object

In [16]:
adata.write_h5ad(outdir + experiment_prefix + "_ATAC.wCisTopic_p2g.h5ad")